🏠 Proyecto: Predicción de Precios de Casas (Ames Housing – OpenML)

In [ ]:
# ✅ 1. Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
# 📥 2. Cargar el dataset desde OpenML
ames = fetch_openml(name="house_prices", as_frame=True)
df = ames.frame

print("Dimensiones:", df.shape)
df.head()

In [ ]:
#3: Variable Objetivo
# Vamos a predecir el precio final de venta
target = "SalePrice"

In [ ]:
#4: Limpieza de datos

# 1. Eliminar columnas con más del 30% de nulos (y hacer copia)
umbral_nulos = 0.3
df = df.loc[:, df.isnull().mean() < umbral_nulos].copy()  # ← Agregamos .copy()

# 2. Rellenar categóricas con la moda
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].fillna(df[col].mode()[0])

# 3. Rellenar numéricas con la mediana
for col in df.select_dtypes(include=["int64", "float64"]).columns:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
#5: Codificación y separación

# One-hot encoding para las variables categóricas
df = pd.get_dummies(df, drop_first=True)

# Separar X e y
X = df.drop(columns=target)
y = df[target]

In [ ]:
#6. División de datos y escalado

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#7. Selección automática de features
selector = SelectKBest(score_func=f_regression, k=50)
X_train_sel = selector.fit_transform(X_train_scaled, y_train)
X_test_sel = selector.transform(X_test_scaled)

In [ ]:
#8. Entrenamiento y evaluación de modelos

modelos = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0)
}

resultados = {}

for nombre, modelo in modelos.items():
    modelo.fit(X_train_sel, y_train)
    pred = modelo.predict(X_test_sel)

    resultados[nombre] = {
        "MAE": mean_absolute_error(y_test, pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, pred)),
        "R2": r2_score(y_test, pred)
    }

In [ ]:
#9. Visualización de resultados

df_resultados = pd.DataFrame(resultados).T
df_resultados.plot(kind="bar", figsize=(10, 5))
plt.title("Comparación de Modelos")
plt.ylabel("Error")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.tight_layout()
plt.show()

df_resultados

In [ ]:
#10. Analisis visual del rendimiento de los modelos

import matplotlib.pyplot as plt
import seaborn as sns

# Obtener predicciones para análisis visual
y_pred_rf = modelos["Random Forest"].predict(X_test_sel)
y_pred_lr = modelos["Linear Regression"].predict(X_test_sel)
y_pred_xgb = modelos["XGBoost"].predict(X_test_sel)

# === 1. Comparar valores reales vs. predichos ===
plt.figure(figsize=(6, 6))
sns.scatterplot(x=y_test, y=y_pred_rf, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Precio Real")
plt.ylabel("Precio Predicho")
plt.title("Random Forest - Precio Real vs. Predicho")
plt.grid(True)
plt.show()

In [ ]:
# === 2. Distribución de los errores (residuos) ===
errores_rf = y_test - y_pred_rf

plt.figure(figsize=(6, 4))
sns.histplot(errores_rf, bins=30, kde=True)
plt.title("Distribución de errores - Random Forest")
plt.xlabel("Error (real - predicho)")
plt.axvline(0, color='red', linestyle='--')
plt.grid(True)
plt.show()

In [ ]:
# === 3. Gráfico de residuos vs. valor predicho ===
plt.figure(figsize=(6, 4))
sns.scatterplot(x=y_pred_rf, y=errores_rf, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Precio Predicho")
plt.ylabel("Error (real - predicho)")
plt.title("Residuos vs. Predicción - Random Forest")
plt.grid(True)
plt.show()

🔚 Conclusiones:
En este proyecto se compararon tres modelos de regresión multivariable aplicados al dataset Ames Housing, evaluando su capacidad para predecir el precio de las viviendas.

Luego del preprocesamiento, selección de características y entrenamiento, los resultados mostraron que:

Random Forest fue el modelo con mejor rendimiento global, logrando el menor error (RMSE) y el mayor coeficiente de determinación (R²).

XGBoost obtuvo resultados similares, siendo una excelente alternativa, especialmente en contextos con mayor volumen de datos.

Regresión Lineal presentó el rendimiento más bajo, aunque sigue siendo valiosa como punto de partida para comparar modelos más complejos.

Los gráficos de análisis de rendimiento (precio real vs. predicho, distribución de errores y residuos vs. predicción) confirmaron la estabilidad y precisión del modelo Random Forest, sin sesgos evidentes ni errores sistemáticos.

Este pipeline puede mejorarse y ampliarse con:

Optimización de hiperparámetros (por ejemplo, mediante GridSearchCV o RandomizedSearchCV),

Validación cruzada para obtener métricas más robustas,

Técnicas de interpretabilidad (como SHAP o importancia de variables),

Exportación del modelo final (joblib o pickle) para su reutilización.